### Court Corners clicker

In [ ]:
import cv2
import numpy as np

# This list will store the clicked points
points = []

def select_points(event, x, y, flags, param):
    """
    Mouse callback function to capture clicks.
    """
    global points
    if event == cv2.EVENT_LBUTTONDOWN:
        points.append((x, y))
        # Draw a circle on the clicked point for visual feedback
        cv2.circle(image, (x, y), 2, (0, 0, 255), -1)
        print(f"Point {len(points)} captured: ({x}, {y})")

# --- Main Script ---
VIDEO_PATH = "VideoInput/video_input2.mp4" # Make sure this path is correct
WINDOW_NAME = "Click to Select Points"



# 1. Load the first frame of the video
cap = cv2.VideoCapture(VIDEO_PATH)
cap.set(cv2.CAP_PROP_POS_FRAMES, 50)
ret, image = cap.read()
if not ret:
    print("Failed to load video.")
else:
    image = cv2.resize(image, (960, 540)) # Use your standard resize
    clone = image.copy() # Keep a clean copy of the image

    # 2. Set up the window and mouse callback
    cv2.namedWindow(WINDOW_NAME)
    cv2.setMouseCallback(WINDOW_NAME, select_points)

    # 3. Wait for the user to click 4 points
    print("Please click on the 4 service box corners in this order:")
    print("1. Top-Left -> 2. Top-Right -> 3. Bottom-Left -> 4. Bottom-Right")
    print("Press 'r' to reset points. Press 'q' to quit when done.")
    
    while True:
        cv2.imshow(WINDOW_NAME, image)
        key = cv2.waitKey(1) & 0xFF

        if key == ord('r'): # Reset if you make a mistake
            image = clone.copy()
            points = []
            print("Points reset.")

        elif key == ord('q') or len(points) == 4:
            break
            
    cv2.destroyAllWindows()
    cap.release()

    # 4. Print the final result in the required format
    if len(points) == 4:
        print("\nSUCCESS! Copy this into your main script:")
        src_points = np.float32(points)
        print(f"src_points = np.float32({src_points.tolist()})")
    else:
        print("\nDid not select 4 points. Please try again.")

### Simple line detector
focuses on central region

In [ ]:
#Court Detection 
import cv2
import numpy as np

# Load video
video_path = "VideoInput/video_input2.mp4"
cap = cv2.VideoCapture(video_path)


while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Reset lines a each frame
    court_lines = []
    
    # Resize for speed
    frame = cv2.resize(frame, (960, 540))
    
    #Preprocess the frame
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # Edge detection
    edges = cv2.Canny(blur, 50, 150)

    # Line detection using Hough Transform
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=80, minLineLength=40, maxLineGap=25)
    
    #draw limit lines
    cv2.line(frame, (170, 130), (170, 445), (255, 0, 0), 1)
    cv2.line(frame, (170, 445), (790, 445), (255, 0, 0), 1)
    
    cv2.line(frame, (170, 130), (790, 130), (255, 0, 0), 1)
    cv2.line(frame, (790, 130), (790, 445), (255, 0, 0), 1)
    
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            
            # Compute line angle in degrees
            if max(y1, y2) > 445 or min(y1, y2) < 130:
                continue
            if max(x1, x2) > 790 or min(x1, x2) < 170:
                continue
            
            angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
            if -15 < angle < 15:
                cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                court_lines.append(lines)
            elif abs(angle) > 45 and abs(angle) < 85:
                cv2.line(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                court_lines.append(line)
            
            """ # Filter: keep near-horizontal (0° ± 10°) or vertical (90° ± 10°) lines
            if not ((-20 <= angle <= 10) or (70 <= angle <= 110)):
                continue
            
            # Filter: length threshold (optional)
            length = np.hypot(x2 - x1, y2 - y1)
            if length < 100:
                continue"""

    cv2.imshow("Court Line Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

### Line stabilizer

In [30]:
from collections import deque

class LineStabilizer:
    """
    A class to smooth detected lines over several frames.
    """
    def __init__(self, buffer_size=5):
        # A deque is a list-like container with fast appends and pops from either end.
        self.buffer = deque(maxlen=buffer_size)
        
    def add_lines(self, lines):
        """Adds the lines from a new frame to the buffer."""
        if lines is not None:
            self.buffer.append(lines)
            
    def get_stable_lines(self):
        """Averages the lines in the buffer to get a stable result."""
        # Combine all lines from all frames in the buffer into one big list
        all_lines = [line for frame_lines in self.buffer for line in frame_lines]
        
        if not all_lines:
            return None
            
        # --- We use the same merging logic as before, but on a larger, more stable dataset ---
        horizontal= []
        for line in all_lines:
            x1, y1, x2, y2 = line
            angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
            if -15 < angle < 15:
                horizontal.append(line)
        
        
        # === 1. Merge Horizontal Lines ===
        if horizontal is not None and len(horizontal) > 0:
            # Sort lines by their y-coordinate to process them from top to bottom
            horizontal.sort(key=lambda line: line[1])
            merged_lines = []
            while len(horizontal) > 0:
                base_line = horizontal.pop(0)
                y_base = (base_line[1] + base_line[3]) / 2
                
                group = [base_line]
                remaining_lines = []
                for line in horizontal:
                    y_curr = (line[1] + line[3]) / 2
                    if abs(y_curr - y_base) < 20:
                        group.append(line)
                    else:
                        remaining_lines.append(line)
                horizontal = remaining_lines
                
                x_coords = np.array([line[0] for line in group] + [line[2] for line in group])
                y_coords = np.array([line[1] for line in group] + [line[3] for line in group])
                
                avg_y = int(np.mean(y_coords))
                min_x = int(np.min(x_coords))
                max_x = int(np.max(x_coords))
                
                merged_lines.append([min_x, avg_y, max_x, avg_y])
        return merged_lines

In [31]:
import cv2
import numpy as np

video_path = "VideoInput/video_input2.mp4"
cap = cv2.VideoCapture(video_path)

# Use your previously found source points
src_points = np.float32([[288.0, 152.0], [668.0, 150.0], [182.0, 429.0], [783.0, 428.0]])
width, height = 400, 500
dst_points = np.float32([[0, 0], [width, 0], [0, height], [width, height]])
M = cv2.getPerspectiveTransform(src_points, dst_points)

# Initialize the stabilizer before the loop
line_stabilizer = LineStabilizer(buffer_size=15)

def filter_pixels(gray):
    """
    Filter pixels by using the court line structure
    """
    dist_tau = 3
    intensity_threshold = 40
    for i in range(dist_tau, len(gray) - dist_tau):
      for j in range(dist_tau, len(gray[0]) - dist_tau):
        if gray[i, j] == 0:
            continue
        if (gray[i, j] - gray[i + dist_tau, j] > intensity_threshold and gray[i, j] - gray[i - dist_tau, j] > intensity_threshold):
            continue
        if (gray[i, j] - gray[i, j + dist_tau] > intensity_threshold and gray[i, j] - gray[i, j - dist_tau] > intensity_threshold):
            continue
        gray[i, j] = 0
    return gray

def classify_lines(lines):
    """
    Classify lines into horizontal and vertical based on their angle.
    """
    horizontal = []
    vertical = []
    
    for line in lines:
        x1, y1, x2, y2 = line
        angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
        
        if -15 < angle < 15:  # Horizontal line
            horizontal.append((x1, y1, x2, y2))
        elif 45 < angle < 100 or -100 < angle < -45:  # Vertical line
            vertical.append((x1, y1, x2, y2))
    
    return horizontal, vertical

def merge_lines(horizontal):
    new_horizontal= []
    
    # === 1. Merge Horizontal Lines ===
    if horizontal is not None and len(horizontal) > 0:
        # Sort lines by their y-coordinate to process them from top to bottom
        horizontal.sort(key=lambda line: line[1])
        
        while len(horizontal) > 0:
            base_line = horizontal.pop(0)
            y_base = (base_line[1] + base_line[3]) / 2
            
            group, horizontal = [base_line], [line for line in horizontal if not (abs(((line[1] + line[3]) / 2) - y_base) < 20)]
            all_x = np.array([p for line in group for p in (line[0], line[2])])
            all_y = np.array([p for line in group for p in (line[1], line[3])])
            new_horizontal.append([int(np.min(all_x)), int(np.mean(all_y)), int(np.max(all_x)), int(np.mean(all_y))])
    return new_horizontal

def detect_lines(frame):
    # Detect all lines
    lines = cv2.HoughLinesP(gray, 1, np.pi / 180, 80, minLineLength=50, maxLineGap=20)
    lines = np.squeeze(lines)
    
    horizontal, vertical = classify_lines(lines)
    
    # 2. Add raw lines to our stabilizer
    line_stabilizer.add_lines(horizontal)
    
    # 3. Get the STABLE lines by averaging over the buffer
    horizontal = line_stabilizer.get_stable_lines()
    #horizontal = merge_lines(horizontal)
    
    return horizontal, vertical

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (960, 540))
    
    # 1. Get binary image
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)[1]
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    edges = cv2.Canny(gray, 75, 150)
    
    horizontal, vertical = detect_lines(edges)
    
    for line in horizontal:
        x1, y1, x2, y2 = line[0]
        cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)
    
    for line in vertical:
        x1, y1, x2, y2 = line
        cv2.line(frame, (x1, y1), (x2, y2), (0, 0, 255), 1)
    
    #warped_frame = cv2.warpPerspective(frame, M, (width, height))
    cv2.imshow("Court Line Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
 

cap.release()
cv2.destroyAllWindows()
print("Processing complete.")

TypeError: cannot unpack non-iterable int object

### Calibration

In [ ]:
import cv2
import numpy as np

class Calibration:
    """
    A class to handle the interactive calibration process where a user
    draws a rectangle to define a Region of Interest (ROI).
    """
    def __init__(self, frame):
        """
        Initializes the Calibration object.
        Args:
            frame (np.array): The first frame of the video for the user to draw on.
        """
        self.frame = frame.copy()
        self.roi_points = []
        self.drawing = False
        self.roi_rect = None
        self.bottom_service_line = None

    def _mouse_handler(self, event, x, y, flags, param):
        """
        Internal mouse handler for drawing the rectangle.
        """
        if event == cv2.EVENT_LBUTTONDOWN:
            self.drawing = True
            self.roi_points = [(x, y)]

        elif event == cv2.EVENT_MOUSEMOVE and self.drawing:
            frame_copy = self.frame.copy()
            cv2.rectangle(frame_copy, self.roi_points[0], (x, y), (0, 255, 0), 2)
            cv2.imshow("Calibration", frame_copy)

        elif event == cv2.EVENT_LBUTTONUP:
            self.drawing = False
            self.roi_points.append((x, y))
            frame_copy = self.frame.copy()
            cv2.rectangle(frame_copy, self.roi_points[0], self.roi_points[1], (0, 255, 0), 2)
            cv2.imshow("Calibration", frame_copy)

    def _merge_lines(self, lines):
        """
        Merges a list of horizontal line segments into a single line.
        """
        if not lines:
            return None
        
        # Flatten the list of lines to get all x and y coordinates
        all_x = [p[0] for p in lines] + [p[2] for p in lines]
        all_y = [p[1] for p in lines] + [p[3] for p in lines]
        
        # Calculate the start and end points of the merged line
        min_x = min(all_x)
        max_x = max(all_x)
        avg_y = int(np.mean(all_y))
        
        return ((min_x, avg_y), (max_x, avg_y))

    def get_roi(self):
        """
        Starts the calibration UI, waits for user input, and returns the ROI.

        Returns:
            tuple: A tuple (x1, y1, x2, y2) representing the ROI rectangle,
                   or None if calibration is cancelled.
        """
        print("--- Court Calibration ---")
        print("Click and drag to draw a box around the bottom service line.")
        print("After drawing the box, press 'c' to start processing.")
        
        cv2.imshow("Calibration", self.frame)
        cv2.setMouseCallback("Calibration", self._mouse_handler)
        
        key = cv2.waitKey(0)
        
        cv2.destroyWindow("Calibration")

        if key == ord('c') and len(self.roi_points) == 2:
            # Ensure x1 < x2 and y1 < y2
            x1 = min(self.roi_points[0][0], self.roi_points[1][0])
            y1 = min(self.roi_points[0][1], self.roi_points[1][1])
            x2 = max(self.roi_points[0][0], self.roi_points[1][0])
            y2 = max(self.roi_points[0][1], self.roi_points[1][1])
            self.roi_rect = (x1, y1, x2, y2)
            print("Calibration successful.")
            return self.roi_rect
        else:
            print("Calibration cancelled or incomplete.")
            return None
        
    def run(self):
        """
        Gets the ROI and then performs line detection within it, displaying the result.
        """
        roi_rect = self.get_roi()
        if roi_rect is None:
            return

        x1, y1, x2, y2 = roi_rect
        roi = self.frame[y1:y2, x1:x2]
        
        # --- Improved image processing pipeline ---
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        # Apply a Gaussian blur to reduce noise and improve Canny performance
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        # Apply Canny edge detection directly on the grayscale (blurred) image
        edges = cv2.Canny(blurred, 50, 150)
        
        lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=30, minLineLength=20, maxLineGap=15)
        
        horizontal_lines = []
        if lines is not None:
            for line in lines:
                lx1, ly1, lx2, ly2 = line[0]
                # Filter for lines that are nearly horizontal
                angle = np.degrees(np.arctan2(ly2 - ly1, lx2 - lx1))
                if abs(angle) < 15:
                    horizontal_lines.append((lx1, ly1, lx2, ly2))
        
        # Merge the detected horizontal lines into one
        merged_line = self._merge_lines(horizontal_lines)
        self.bottom_service_line = merged_line
        
        # Draw the final, clean line on the ROI
        if merged_line:
            p1, p2 = merged_line
            cv2.line(roi, p1, p2, (0, 255, 0), 1)
        
        cv2.imshow("ROI with Detected Lines", roi)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

if __name__ == "__main__":
    # Example usage:
    video_path = "VideoInput/video_input2.mp4"
    cap = cv2.VideoCapture(video_path)
    
    ret, first_frame = cap.read()
    if not ret:
        print("Error: Could not read the first frame.")
        exit()
    first_frame = cv2.resize(first_frame, (960, 540))

    c = Calibration(first_frame)
    c.run()
    
    cap.release()
    print(c.bottom_service_line)

### V1

In [50]:
from collections import deque

class LineStabilizer2:
    """
    A class to smooth detected lines over several frames.
    """
    def __init__(self, buffer_size=5):
        # A deque is a list-like container with fast appends and pops from either end.
        self.buffer = deque(maxlen=buffer_size)
        
    def add_lines(self, lines):
        """Adds the lines from a new frame to the buffer."""
        if lines is not None:
            self.buffer.append(lines)
            
    def get_stable_lines(self):
        """Averages the lines in the buffer to get a stable result."""
        # Combine all lines from all frames in the buffer into one big list
        all_lines = [line for frame_lines in self.buffer for line in frame_lines]
        
        if not all_lines:
            return None
            
        # --- We use the same merging logic as before, but on a larger, more stable dataset ---
        horizontal= []
        for line in all_lines:
            x1, y1, x2, y2 = line
            angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
            if -15 < angle < 15:
                horizontal.append(line)
        
        
        # === 1. Merge Horizontal Lines ===
        if horizontal is not None and len(horizontal) > 0:
            # Sort lines by their y-coordinate to process them from top to bottom
            horizontal.sort(key=lambda line: line[1])
            merged_lines = []
            while len(horizontal) > 0:
                base_line = horizontal.pop(0)
                y_base = (base_line[1] + base_line[3]) / 2
                x1_base = base_line[0]
                x2_base = base_line[2]
                
                group = [base_line]
                remaining_lines = []
                for line in horizontal:
                    y_curr = (line[1] + line[3]) / 2
                    if abs(y_curr - y_base) < 10 and (line[0] < x2_base + 20 and line[2] > x1_base - 20):
                        group.append(line)
                    else:
                        remaining_lines.append(line)
                horizontal = remaining_lines
                
                x_coords = np.array([line[0] for line in group] + [line[2] for line in group])
                y_coords = np.array([line[1] for line in group] + [line[3] for line in group])
                
                avg_y = int(np.mean(y_coords))
                min_x = int(np.min(x_coords))
                max_x = int(np.max(x_coords))
                
                merged_lines.append([min_x, avg_y, max_x, avg_y])
        return merged_lines

In [51]:
class Court():
    def __init__(self):
        self.service_line_angle = 0
        self.lines = {
            'top_baseline': None,
            'bottom_baseline': None, 
            'top_service_line': None,
            'bottom_service_line': None,
            'left_singles_line': None, 
            'right_singles_line': None,
            'left_doubles_line': None,
            'right_doubles_line': None
        }


In [52]:
def classify_lines(lines):
    """
    Classify lines into horizontal and vertical based on their angle.
    """
    horizontal = []
    vertical = []
    
    for line in lines:
        x1, y1, x2, y2 = line
        angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
        
        if -15 < angle < 15:  # Horizontal line
            horizontal.append((x1, y1, x2, y2))
        elif 45 < angle < 100 or -100 < angle < -45:  # Vertical line
            vertical.append((x1, y1, x2, y2))
    
    return horizontal, vertical

def merge_lines(horizontal):
    new_horizontal= []
    
    if horizontal is not None and len(horizontal) > 0:
        horizontal.sort(key=lambda line: line[1])
        
        while len(horizontal) > 0:
            base_line = horizontal.pop(0)
            y_base = (base_line[1] + base_line[3]) / 2
            
            group, horizontal = [base_line], [line for line in horizontal if not (abs(((line[1] + line[3]) / 2) - y_base) < 5)]
            all_x = np.array([p for line in group for p in (line[0], line[2])])
            all_y = np.array([p for line in group for p in (line[1], line[3])])
            new_horizontal.append([int(np.min(all_x)), int(np.mean(all_y)), int(np.max(all_x)), int(np.mean(all_y))])
    return new_horizontal

def detect_lines(gray):
    # Detect all lines
    lines = cv2.HoughLinesP(gray, 1, np.pi / 180, 80, minLineLength=90, maxLineGap=20)
    lines = np.squeeze(lines)
    
    unstable_horizontal, stable_vertical = classify_lines(lines)
    
    line_stabilizer.add_lines(unstable_horizontal)
    stable_horizontal = line_stabilizer.get_stable_lines()
    
    #horizontal = merge_lines(horizontal)
    
    return stable_horizontal, stable_vertical

In [53]:
import cv2
import numpy as np

video_path = "VideoInput/video_input2.mp4"
cap = cv2.VideoCapture(video_path)

# Use your previously found source points
src_points = np.float32([[288.0, 152.0], [668.0, 150.0], [182.0, 429.0], [783.0, 428.0]])
width, height = 400, 500
dst_points = np.float32([[0, 0], [width, 0], [0, height], [width, height]])
M = cv2.getPerspectiveTransform(src_points, dst_points)

# Initialize the stabilizer before the loop
line_stabilizer = LineStabilizer2(buffer_size=15)
court = Court()
                
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (960, 540))
    
    # 1. Get binary image
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #gray = cv2.threshold(frame, 200, 255, cv2.THRESH_BINARY)[1]
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    
    horizontal, vertical = detect_lines(edges)
    
    
    for line in horizontal:
        x1, y1, x2, y2 = line
        cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)
    
    for line in vertical:
        x1, y1, x2, y2 = line
        cv2.line(frame, (x1, y1), (x2, y2), (0, 0, 255), 1)
        
    #warped_frame = cv2.warpPerspective(frame, M, (width, height))
    cv2.imshow("Court Line Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
 

cap.release()
cv2.destroyAllWindows()
print("Processing complete.")

Processing complete.


### homograft

In [54]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


class CourtReference:
    """
    Court reference model
    """
    def __init__(self):
        self.baseline_top = ((286, 561), (1379, 561))
        self.baseline_bottom = ((286, 2935), (1379, 2935))
        self.net = ((286, 1748), (1379, 1748))
        self.left_court_line = ((286, 561), (286, 2935))
        self.right_court_line = ((1379, 561), (1379, 2935))
        self.left_inner_line = ((423, 561), (423, 2935))
        self.right_inner_line = ((1242, 561), (1242, 2935))
        self.middle_line = ((832, 1110), (832, 2386))
        self.top_inner_line = ((423, 1110), (1242, 1110))
        self.bottom_inner_line = ((423, 2386), (1242, 2386))
        self.top_extra_part = (832.5, 580)
        self.bottom_extra_part = (832.5, 2910)

        self.court_conf = {1: [*self.baseline_top, *self.baseline_bottom],
                           2: [self.left_inner_line[0], self.right_inner_line[0], self.left_inner_line[1],
                               self.right_inner_line[1]],
                           3: [self.left_inner_line[0], self.right_court_line[0], self.left_inner_line[1],
                               self.right_court_line[1]],
                           4: [self.left_court_line[0], self.right_inner_line[0], self.left_court_line[1],
                               self.right_inner_line[1]],
                           5: [*self.top_inner_line, *self.bottom_inner_line],
                           6: [*self.top_inner_line, self.left_inner_line[1], self.right_inner_line[1]],
                           7: [self.left_inner_line[0], self.right_inner_line[0], *self.bottom_inner_line],
                           8: [self.right_inner_line[0], self.right_court_line[0], self.right_inner_line[1],
                               self.right_court_line[1]],
                           9: [self.left_court_line[0], self.left_inner_line[0], self.left_court_line[1],
                               self.left_inner_line[1]],
                           10: [self.top_inner_line[0], self.middle_line[0], self.bottom_inner_line[0],
                                self.middle_line[1]],
                           11: [self.middle_line[0], self.top_inner_line[1], self.middle_line[1],
                                self.bottom_inner_line[1]],
                           12: [*self.bottom_inner_line, self.left_inner_line[1], self.right_inner_line[1]]}
        self.line_width = 1
        self.court_width = 1117
        self.court_height = 2408
        self.top_bottom_border = 549
        self.right_left_border = 274
        self.court_total_width = self.court_width + self.right_left_border * 2
        self.court_total_height = self.court_height + self.top_bottom_border * 2

        self.court = cv2.cvtColor(cv2.imread('old/court_configurations/court_reference.png'), cv2.COLOR_BGR2GRAY)

    def build_court_reference(self):
        """
        Create court reference image using the lines positions
        """
        court = np.zeros((self.court_height + 2 * self.top_bottom_border, self.court_width + 2 * self.right_left_border), dtype=np.uint8)
        cv2.line(court, *self.baseline_top, 1, self.line_width)
        cv2.line(court, *self.baseline_bottom, 1, self.line_width)
        # cv2.line(court, *self.net, 1, self.line_width)
        cv2.line(court, *self.top_inner_line, 1, self.line_width)
        cv2.line(court, *self.bottom_inner_line, 1, self.line_width)
        cv2.line(court, *self.left_court_line, 1, self.line_width)
        cv2.line(court, *self.right_court_line, 1, self.line_width)
        cv2.line(court, *self.left_inner_line, 1, self.line_width)
        cv2.line(court, *self.right_inner_line, 1, self.line_width)
        cv2.line(court, *self.middle_line, 1, self.line_width)
        court = cv2.dilate(court, np.ones((5, 5), dtype=np.uint8))
        plt.imsave('old/court_configurations/court_reference.png', court, cmap='gray')
        self.court = court
        return court

    def get_important_lines(self):
        """
        Returns all lines of the court
        """
        lines = [*self.baseline_top, *self.baseline_bottom, *self.net, *self.left_court_line, *self.right_court_line,
                 *self.left_inner_line, *self.right_inner_line, *self.middle_line,
                 *self.top_inner_line, *self.bottom_inner_line]
        return lines

    def get_extra_parts(self):
        parts = [self.top_extra_part, self.bottom_extra_part]
        return parts

    def save_all_court_configurations(self):
        """
        Create all configurations of 4 points on court reference
        """
        for i, conf in self.court_conf.items():
            c = cv2.cvtColor(255 - self.court, cv2.COLOR_GRAY2BGR)
            for p in conf:
                c = cv2.circle(c, p, 15, (0, 0, 255), 30)
            cv2.imwrite(f'old/court_configurations/court_conf_{i}.png', c)

    def get_court_mask(self, mask_type=0):
        """
        Get mask of the court
        """
        mask = np.ones_like(self.court)
        if mask_type == 1:  # Bottom half court
            mask[:self.net[0][1] - 1000, :] = 0
        elif mask_type == 2:  # Top half court
            mask[self.net[0][1]:, :] = 0
        elif mask_type == 3: # court without margins
            mask[:self.baseline_top[0][1], :] = 0
            mask[self.baseline_bottom[0][1]:, :] = 0
            mask[:, :self.left_court_line[0][0]] = 0
            mask[:, self.right_court_line[0][0]:] = 0
        return mask


if __name__ == '__main__':
    c = CourtReference()
    c.build_court_reference()

In [55]:
def get_confi_score(matrix, court_reference, frame, gray):
    """
    Calculate transformation score
    """
    court = cv2.warpPerspective(court_reference.court, matrix, frame.shape[1::-1])
    court[court > 0] = 1
    gray = gray.copy()
    gray[gray > 0] = 1
    correct = court * gray
    wrong = court - correct
    c_p = np.sum(correct)
    w_p = np.sum(wrong)
    return c_p - 0.5 * w_p

In [56]:
best_conf = 0
court_reference = CourtReference()
from sympy import Line
from itertools import combinations
def line_intersection(line1, line2):
    """
    Find 2 lines intersection point
    """
    l1 = Line(line1[0], line1[1])
    l2 = Line(line2[0], line2[1])

    intersection = l1.intersection(l2)
    return intersection[0].coordinates

def sort_intersection_points(intersections):
    """
    sort intersection points from top left to bottom right
    """
    y_sorted = sorted(intersections, key=lambda x: x[1])
    p12 = y_sorted[:2]
    p34 = y_sorted[2:]
    p12 = sorted(p12, key=lambda x: x[0])
    p34 = sorted(p34, key=lambda x: x[0])
    return p12 + p34

def find_homography(horizontal_lines, vertical_lines, court_reference, frame, gray):
        """
        Finds transformation from reference court to frame`s court using 4 pairs of matching points
        """
        max_score = -np.inf
        max_mat = None
        max_inv_mat = None
        k = 0
        # Loop over every pair of horizontal lines and every pair of vertical lines
        for horizontal_pair in list(combinations(horizontal_lines, 2)):
            for vertical_pair in list(combinations(vertical_lines, 2)):
                h1, h2 = horizontal_pair
                v1, v2 = vertical_pair
                # Finding intersection points of all lines
                i1 = line_intersection((tuple(h1[:2]), tuple(h1[2:])), (tuple(v1[0:2]), tuple(v1[2:])))
                i2 = line_intersection((tuple(h1[:2]), tuple(h1[2:])), (tuple(v2[0:2]), tuple(v2[2:])))
                i3 = line_intersection((tuple(h2[:2]), tuple(h2[2:])), (tuple(v1[0:2]), tuple(v1[2:])))
                i4 = line_intersection((tuple(h2[:2]), tuple(h2[2:])), (tuple(v2[0:2]), tuple(v2[2:])))

                intersections = [i1, i2, i3, i4]
                intersections = sort_intersection_points(intersections)

                for i, configuration in court_reference.court_conf.items():
                    # Find transformation
                    matrix, _ = cv2.findHomography(np.float32(configuration), np.float32(intersections), method=0)
                    inv_matrix = cv2.invert(matrix)[1]
                    # Get transformation score
                    confi_score = get_confi_score(matrix, court_reference, frame, gray)

                    if max_score < confi_score:
                        max_score = confi_score
                        max_mat = matrix
                        max_inv_mat = inv_matrix
                        best_conf = i

                    k += 1

        return max_mat, max_inv_mat, max_score

In [ ]:
import cv2
import numpy as np

video_path = "VideoInput/video_input2.mp4"
cap = cv2.VideoCapture(video_path)

# Use your previously found source points
src_points = np.float32([[288.0, 152.0], [668.0, 150.0], [182.0, 429.0], [783.0, 428.0]])
width, height = 400, 500
dst_points = np.float32([[0, 0], [width, 0], [0, height], [width, height]])
M = cv2.getPerspectiveTransform(src_points, dst_points)

# Initialize the stabilizer before the loop
line_stabilizer = LineStabilizer2(buffer_size=15)
court = Court()
                
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (960, 540))
    
    # 1. Get binary image
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)[1]
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    
    horizontal, vertical = detect_lines(edges)
    
    court_warp_matrix, game_warp_matrix, court_score = find_homography(horizontal, vertical, court_reference, frame, gray)
    print(court_warp_matrix, game_warp_matrix, court_score)
    for line in horizontal:
        x1, y1, x2, y2 = line[0]
        cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)
    
    for line in vertical:
        x1, y1, x2, y2 = line
        cv2.line(frame, (x1, y1), (x2, y2), (0, 0, 255), 1)
        
    #warped_frame = cv2.warpPerspective(frame, M, (width, height))
    cv2.imshow("Court Line Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
 

cap.release()
cv2.destroyAllWindows()
print("Processing complete.")